In [1]:
from epymorph.data_shape import Shapes
from epymorph.geo.adrio import adrio_maker_library
from epymorph.geo.dynamic import DynamicGeo
from epymorph.geo.spec import DynamicGeoSpec, Year
from epymorph.geography.us_census import CountyScope, StateScope
from epymorph.simulation import geo_attrib


spec = DynamicGeoSpec(
    attributes=[
        geo_attrib('label', str, Shapes.N),
        geo_attrib('population', int, Shapes.N),
        # geo_attrib("covid_cases_per_100k", int, Shapes.TxN),
        # geo_attrib("covid_hospitalizations_per_100k", int, Shapes.TxN),
        geo_attrib("covid_hospitalization_avg_facility", float, Shapes.TxN),
        geo_attrib("covid_hospitalization_sum_facility", int, Shapes.TxN),
        geo_attrib("influenza_hospitalization_avg_facility", float, Shapes.TxN),
        geo_attrib("influenza_hospitalization_sum_facility", int, Shapes.TxN),
        geo_attrib("covid_hospitalization_avg_state", float, Shapes.TxN),
        geo_attrib("covid_hospitalization_sum_state", int, Shapes.TxN),
        geo_attrib("influenza_hospitalization_avg_state", float, Shapes.TxN),
        geo_attrib("influenza_hospitalization_sum_state", int, Shapes.TxN),
        geo_attrib("full_covid_vaccinations", int, Shapes.TxN),
        geo_attrib("one_dose_covid_vaccinations", int, Shapes.TxN),
        geo_attrib("covid_booster_doses", int, Shapes.TxN),
        geo_attrib("covid_deaths_county", int, Shapes.TxN)
    ],
    time_period=Year(2022),
    # scope=CountyScope.in_counties(['04007', '04013']),
    # scope=CountyScope.in_states(['04', '08']),
    scope=StateScope.in_states(['04', '08']),
    source={
        'label': 'Census:name',
        'population': 'Census',
        "covid_cases_per_100k": "CDC",
        "covid_hospitalizations_per_100k": "CDC",
        "covid_hospitalization_avg_facility": "CDC",
        "covid_hospitalization_sum_facility": "CDC",
        "influenza_hospitalization_avg_facility": "CDC",
        "influenza_hospitalization_sum_facility": "CDC",
        "covid_hospitalization_avg_state": "CDC",
        "covid_hospitalization_sum_state": "CDC",
        "influenza_hospitalization_avg_state": "CDC",
        "influenza_hospitalization_sum_state": "CDC",
        "full_covid_vaccinations": "CDC",
        "one_dose_covid_vaccinations": "CDC",
        "covid_booster_doses": "CDC",
        "covid_deaths_county": "CDC"
    }
)

geo = DynamicGeo.from_library(spec, adrio_maker_library)

# geo.fetch_all()
# geo['covid_cases_per_100k']
geo['covid_hospitalization_avg_state']
# geo['full_covid_vaccinations']
# geo['covid_deaths']

array([[279., 236.],
       [356., 330.],
       [450., 346.],
       [512., 298.],
       [502., 257.],
       [403., 178.],
       [303., 124.],
       [240.,  86.],
       [161.,  63.],
       [120.,  40.],
       [ 95.,  33.],
       [ 66.,  21.],
       [ 58.,  19.],
       [ 44.,  18.],
       [ 47.,  14.],
       [ 37.,  16.],
       [ 36.,  26.],
       [ 35.,  26.],
       [ 42.,  36.],
       [ 49.,  37.],
       [ 57.,  49.],
       [ 60.,  54.],
       [ 81.,  69.],
       [ 87.,  91.],
       [ 93.,  81.],
       [106.,  82.],
       [102.,  84.],
       [106.,  97.],
       [120.,  90.],
       [119.,  85.],
       [107.,  73.],
       [ 97.,  71.],
       [ 96.,  63.],
       [ 88.,  53.],
       [ 63.,  51.],
       [ 61.,  50.],
       [ 52.,  44.],
       [ 44.,  46.],
       [ 40.,  38.],
       [ 40.,  39.],
       [ 38.,  34.],
       [ 34.,  38.],
       [ 48.,  43.],
       [ 47.,  54.],
       [ 67.,  71.],
       [ 96.,  85.],
       [120.,  99.],
       [147.,